# Sequence-to-Sequence Problem
In this section, we define a contrived and scalable sequence-to-sequence prediction problem.

The source sequence is a series of randomly generated integer values, such as [20, 36, 40, 10, 34, 28], and the target sequence is a reversed pre-defined subset of the input sequence, such as the first 3 elements in reverse order [40, 36, 20].

The length of the source sequence is configurable; so is the cardinality of the input and output sequence and the length of the target sequence.

We will use source sequences of 6 elements, a cardinality of 50, and target sequences of 3 elements.

Below are some more examples to make this concrete.

Source,=======================Target      <br/>
[13, 28, 18, 7, 9, 5] ==============> [18, 28, 13]<br/>
[29, 44, 38, 15, 26, 22] ============>[38, 44, 29]<br/>
[27, 40, 31, 29, 32, 1] =============>[31, 40, 27]<br/>


In [1]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
import numpy as np
print('Imported')

Using TensorFlow backend.


Imported


In [15]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]

# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
        X1, X2, y = list(), list(), list()
        for _ in range(n_samples):
                # generate source sequence
                source = generate_sequence(n_in, cardinality)
                # define padded target sequence
                target = source[:n_out]
                target.reverse()
                # create padded input target sequence
                target_in = [0] + target[:-1]
                # encode
                src_encoded = to_categorical([source], num_classes=cardinality)
                tar_encoded = to_categorical([target], num_classes=cardinality)
                tar2_encoded = to_categorical([target_in], num_classes=cardinality)
                # store
                X1.append(src_encoded)
                X2.append(tar2_encoded)
                y.append(tar_encoded)
        X1 = np.squeeze(array(X1), axis=1) 
        X2 = np.squeeze(array(X2), axis=1) 
        y = np.squeeze(array(y), axis=1) 
        return X1, X2, y

# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
	# define training encoder
	encoder_inputs = Input(shape=(None, n_input))
	encoder = LSTM(n_units, return_state=True)
	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
	encoder_states = [state_h, state_c]
	# define training decoder
	decoder_inputs = Input(shape=(None, n_output))
	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
	decoder_dense = Dense(n_output, activation='softmax')
	decoder_outputs = decoder_dense(decoder_outputs)
	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
	# define inference encoder
	encoder_model = Model(encoder_inputs, encoder_states)
	# define inference decoder
	decoder_state_input_h = Input(shape=(n_units,))
	decoder_state_input_c = Input(shape=(n_units,))
	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
	decoder_states = [state_h, state_c]
	decoder_outputs = decoder_dense(decoder_outputs)
	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
	# return all models
	return model, encoder_model, decoder_model

# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
	# encode
	state = infenc.predict(source)
	# start of sequence input
	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
	# collect predictions
	output = list()
	for t in range(n_steps):
		# predict next char
		yhat, h, c = infdec.predict([target_seq] + state)
		# store prediction
		output.append(yhat[0,0,:])
		# update state
		state = [h, c]
		# update target sequence
		target_seq = yhat
	return array(output)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [argmax(vector) for vector in encoded_seq]

# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 8000 )
print(X1.shape,X2.shape,y.shape)

(5000, 6, 51) (5000, 6, 51) (5000, 6, 51)


In [16]:
# train model
epochs = 50
history = train.fit([X1, X2], y, epochs=epochs)
acc = (history.history['accuracy'][0]) * 100
acc = round(acc, 1)

Epoch 1/50
5000/5000 [==============================] - 17s 3ms/step - loss: 3.2926 - accuracy: 0.1470
Epoch 2/50
5000/5000 [==============================] - 17s 3ms/step - loss: 2.3010 - accuracy: 0.2630
Epoch 3/50
5000/5000 [==============================] - 12s 2ms/step - loss: 1.7808 - accuracy: 0.4116
Epoch 4/50
5000/5000 [==============================] - 7s 1ms/step - loss: 1.1913 - accuracy: 0.6570
Epoch 5/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.7099 - accuracy: 0.8466
Epoch 6/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.4442 - accuracy: 0.9248
Epoch 7/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.2983 - accuracy: 0.9595
Epoch 8/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.2171 - accuracy: 0.9730
Epoch 9/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1604 - accuracy: 0.9838
Epoch 10/50
5000/5000 [==============================] - 7s 1ms/step - loss: 0.

In [25]:
train.save(str(n_steps_in) + '_in_'+ str(n_steps_out)+'_out_acc_'+str(acc)+'_epochs_'+str(epochs)+'.h5')

In [34]:
# evaluate LSTM
correct = 0
testSamples  = 10
for _ in range(testSamples):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))
        
print("Corret prediction: ", correct, ' out of  ', testSamples)

X=[5, 9, 50, 24, 25, 5] y=[5, 25, 24, 50, 9, 5], yhat=[5, 25, 24, 50, 9, 5]
X=[17, 21, 21, 10, 50, 49] y=[49, 50, 10, 21, 21, 17], yhat=[49, 50, 10, 21, 21, 29]
X=[17, 36, 3, 17, 38, 28] y=[28, 38, 17, 3, 36, 17], yhat=[28, 38, 17, 3, 36, 17]
X=[37, 43, 16, 38, 11, 16] y=[16, 11, 38, 16, 43, 37], yhat=[16, 11, 38, 16, 43, 37]
X=[31, 35, 40, 24, 6, 33] y=[33, 6, 24, 40, 35, 31], yhat=[33, 6, 24, 40, 35, 31]
X=[7, 24, 21, 40, 33, 13] y=[13, 33, 40, 21, 24, 7], yhat=[13, 33, 40, 21, 24, 7]
X=[6, 48, 38, 11, 45, 9] y=[9, 45, 11, 38, 48, 6], yhat=[9, 45, 11, 38, 48, 6]
X=[2, 27, 1, 31, 49, 48] y=[48, 49, 31, 1, 27, 2], yhat=[48, 49, 31, 1, 27, 2]
X=[39, 25, 2, 2, 8, 46] y=[46, 8, 2, 2, 25, 39], yhat=[46, 8, 2, 2, 25, 39]
X=[31, 1, 45, 36, 26, 34] y=[34, 26, 36, 45, 1, 31], yhat=[34, 26, 36, 45, 1, 31]
Corret prediction:  9  out of   10


In [35]:
print('Length: ',len(X1[0]))
print('X1')
print(X1[0])
print('X2')
print(X2[0])
print('y')
print(y[0])
print(y.shape)

Length:  6
X1
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]]
X2
[[1. 0. 0. 0. 0. 0. 0.

In [38]:
print(one_hot_decode(X1[0]))
print(one_hot_decode(y[0]))
print(one_hot_decode(X2[0]))

[31, 1, 45, 36, 26, 34]
[34, 26, 36, 45, 1, 31]
[0, 34, 26, 36, 45, 1]
